# Query Performance

In [60]:
from pymongo import MongoClient
import pandas as pd
import time

In [24]:
client = MongoClient('mongodb://localhost:27017/')
db = client['animeDB']

In [25]:
collec = db.list_collection_names()
collec

['studios_l',
 'genres_l',
 'anime_recommendation_table',
 'demo_l',
 'tm_ky',
 'anime_table',
 'rank_table',
 'anime_ranking_table']

## End-User Queries

### Query 1

In [26]:
# Récupérer l'ID du genre "Adventure" et le convertir en chaîne de caractères
adventure_genre_id = db.genres_l.find_one({"genres_de": "Adventure"})['genres_id']
adventure_genre_id = str(adventure_genre_id)


In [27]:
def execute_and_print_query_1():
    results = db.anime_ranking_table.aggregate([
        {"$addFields": {"genres_id_array": {"$split": ["$genres_id", ","]}}},
        {"$match": {"genres_id_array": adventure_genre_id, "popularity": {"$gt": 500}}},
        {"$project": {"_id": 0, "title": 1, "popularity": 1, "genre": "Adventure"}}
    ])
    for result in results:
        print(result)
execute_and_print_query_1()

{'popularity': 1412, 'title': 'Sunabouzu', 'genre': 'Adventure'}
{'popularity': 737, 'title': 'Black Cat (TV)', 'genre': 'Adventure'}
{'popularity': 2382, 'title': 'Ayashi no Ceres', 'genre': 'Adventure'}
{'popularity': 527, 'title': 'Shaman King', 'genre': 'Adventure'}
{'popularity': 1577, 'title': 'Fushigi Yuugi', 'genre': 'Adventure'}
{'popularity': 3301, 'title': 'Saiyuuki Reload', 'genre': 'Adventure'}
{'popularity': 2541, 'title': 'Gensoumaden Saiyuuki', 'genre': 'Adventure'}
{'popularity': 1818, 'title': 'Scrapped Princess', 'genre': 'Adventure'}
{'popularity': 883, 'title': 'Tsubasa Chronicle', 'genre': 'Adventure'}
{'popularity': 1642, 'title': 'Kyou kara Maou!', 'genre': 'Adventure'}
{'popularity': 2229, 'title': '.hack//Tasogare no Udewa Densetsu', 'genre': 'Adventure'}
{'popularity': 2305, 'title': 'Rekka no Honoo', 'genre': 'Adventure'}
{'popularity': 5475, 'title': 'Mahoujin Guruguru', 'genre': 'Adventure'}
{'popularity': 2080, 'title': 'Gun x Sword', 'genre': 'Adventure'

In [50]:
# Fonction pour exécuter la requête 1
def execute_query_1():
    start_time = time.time()
    db.anime_ranking_table.aggregate([
        {"$addFields": {"genres_id_array": {"$split": ["$genres_id", ","]}}},
        {"$match": {"genres_id_array": adventure_genre_id, "popularity": {"$gt": 500}}},
        {"$project": {"_id": 0, "title": 1, "popularity": 1, "genre": "Adventure"}}
    ])
    return time.time() - start_time

# Mesurer les temps d'exécution pour la requête 1
execution_times_query_1 = []
for _ in range(10):
    execution_time = execute_query_1()
    execution_times_query_1.append(execution_time)

# Retirer le max et le min pour la requête 1
execution_times_query_1.remove(max(execution_times_query_1))
execution_times_query_1.remove(min(execution_times_query_1))

# Calculer la moyenne pour la requête 1
average_time_query_1 = sum(execution_times_query_1) / len(execution_times_query_1)
average_time_query_1 = round(average_time_query_1,3)
print(f"Average execution time for query 1: {average_time_query_1} seconds")

Average execution time for query 1: 0.006 seconds


### Query 2

In [29]:
# Récupérer l'ID du studio "Madhouse"
madhouse_studio_id = db.studios_l.find_one({"studio_de": "Madhouse"})['studio_id']

In [30]:
def execute_and_print_query_2():
    results = db.anime_table.aggregate([
        {"$match": {"studio_id": madhouse_studio_id, "status": "finished_airing"}},
        {"$lookup": {
            "from": "studios_l",
            "localField": "studio_id",
            "foreignField": "studio_id",
            "as": "studio_info"}},
        {"$unwind": "$studio_info"},
        {"$group": {
            "_id": "$title",
            "studio": {"$first": "$studio_info.studio_de"}}},
        {"$project": {
            "_id": 0,
            "title": "$_id",
            "studio": 1,
            "status": 1}}
    ])
    for result in results:
        print(result)

execute_and_print_query_2()

{'studio': 'Madhouse', 'title': 'Pita Ten'}
{'studio': 'Madhouse', 'title': 'Needless'}
{'studio': 'Madhouse', 'title': 'ACCA: 13-ku Kansatsu-ka'}
{'studio': 'Madhouse', 'title': 'Shigurui'}
{'studio': 'Madhouse', 'title': 'Btooom!'}
{'studio': 'Madhouse', 'title': 'Highschool of the Dead'}
{'studio': 'Madhouse', 'title': 'Nana'}
{'studio': 'Madhouse', 'title': 'X'}
{'studio': 'Madhouse', 'title': 'Hunter x Hunter (2011)'}
{'studio': 'Madhouse', 'title': 'Saiunkoku Monogatari'}
{'studio': 'Madhouse', 'title': 'No Guns Life'}
{'studio': 'Madhouse', 'title': 'Claymore'}
{'studio': 'Madhouse', 'title': 'Gungrave'}
{'studio': 'Madhouse', 'title': 'Trigun'}
{'studio': 'Madhouse', 'title': 'Black Lagoon'}
{'studio': 'Madhouse', 'title': 'Chuukan Kanriroku Tonegawa'}
{'studio': 'Madhouse', 'title': 'One Punch Man'}
{'studio': 'Madhouse', 'title': 'Touhai Densetsu Akagi: Yami ni Maiorita Tensai'}
{'studio': 'Madhouse', 'title': 'Beck'}
{'studio': 'Madhouse', 'title': 'RideBack'}
{'studio': 'Ma

In [31]:
def execute_query_2():
    start_time = time.time()
    db.anime_table.aggregate([
        {"$match": {"studio_id": madhouse_studio_id, "status": "finished_airing"}},
        {"$lookup": {
            "from": "studios_l",
            "localField": "studio_id",
            "foreignField": "studio_id",
            "as": "studio_info"}},
        {"$unwind": "$studio_info"},
        {"$group": {
            "_id": "$title",
            "studio": {"$first": "$studio_info.studio_de"}}},
        {"$project": {
            "_id": 0,
            "title": "$_id",
            "studio": 1,
            "status": 1}}
    ])
    return time.time() - start_time

# Mesurer les temps d'exécution pour la requête 2
execution_times_query_2 = []
for _ in range(10):
    execution_time = execute_query_2()
    execution_times_query_2.append(execution_time)

# Retirer le max et le min pour la requête 2
execution_times_query_2.remove(max(execution_times_query_2))
execution_times_query_2.remove(min(execution_times_query_2))

# Calculer la moyenne pour la requête 2
average_time_query_2 = sum(execution_times_query_2) / len(execution_times_query_2)
average_time_query_2 = round(average_time_query_2, 2)
print(f"Average execution time for query 2: {average_time_query_2} seconds")

Average execution time for query 2: 0.52 seconds


### Query 3

In [32]:
def execute_and_print_query_3():
    results = db.anime_ranking_table.aggregate([
        {"$match": {
            "$or": [
                {"title": "Yakitate!! Japan"},
                {"title": "Haikyuu!!"},
                {"title": "Gakuen Alice"},
                {"title": "Magi: The Kingdom of Magic"}
            ]}},
        {"$group": {
            "_id": "$title",
            "rank": {"$first": "$rank"},
            "popularity": {"$first": "$popularity"}}},
        {"$sort": {"rank": 1}},
        {"$project": {
            "_id": 0,
            "title": "$_id",
            "rank": 1,
            "popularity": 1}}
    ])
    for result in results:
        print(result)
execute_and_print_query_3()

{'rank': 131.0, 'popularity': 38, 'title': 'Haikyuu!!'}
{'rank': 278.0, 'popularity': 178, 'title': 'Magi: The Kingdom of Magic'}
{'rank': 649.0, 'popularity': 1816, 'title': 'Yakitate!! Japan'}
{'rank': 1213.0, 'popularity': 1787, 'title': 'Gakuen Alice'}


In [55]:
def execute_query_3():
    start_time = time.time()
    db.anime_ranking_table.aggregate([
        {"$match": {
            "$or": [
                {"title": "Yakitate!! Japan"},
                {"title": "Haikyuu!!"},
                {"title": "Gakuen Alice"},
                {"title": "Magi: The Kingdom of Magic"}
            ]}},
        {"$group": {
            "_id": "$title",
            "rank": {"$first": "$rank"},
            "popularity": {"$first": "$popularity"}}},
        {"$sort": {"rank": 1}},
        {"$project": {
            "_id": 0,
            "title": "$_id",
            "rank": 1,
            "popularity": 1}}
    ])
    return time.time() - start_time

# Mesurer les temps d'exécution pour la requête 3
execution_times_query_3 = []
for _ in range(10):
    execution_time = execute_query_3()
    execution_times_query_3.append(execution_time)

# Retirer le max et le min pour la requête 3
execution_times_query_3.remove(max(execution_times_query_3))
execution_times_query_3.remove(min(execution_times_query_3))

# Calculer la moyenne pour la requête 3
average_time_query_3 = sum(execution_times_query_3) / len(execution_times_query_3)
average_time_query_3 = round(average_time_query_3, 3)
print(f"Average execution time for query 3: {average_time_query_3} seconds")

Average execution time for query 3: 0.006 seconds


### Query 4

In [34]:
def execute_and_print_query_4():
    results = db.anime_ranking_table.aggregate([
        {"$match": {"title": "Trigun"}},
        {"$lookup": {
            "from": "demo_l",
            "localField": "demo_id",
            "foreignField": "demo_id",
            "as": "demo_data"}},
        {"$unwind": {
            "path": "$demo_data",
            "preserveNullAndEmptyArrays": True}},
        {"$group": {
            "_id": "$title",
            "demo_de": {"$first": "$demo_data.demo_de"}}},
        {"$project": {
            "_id": 0,
            "title": "$_id",
            "demo_de": {"$ifNull": ["$demo_de", "No demographic data"]}}}
    ])
    for result in results:
        print(result)
execute_and_print_query_4()

{'title': 'Trigun', 'demo_de': 'Shounen'}


In [56]:
def execute_query_4():
    start_time = time.time()
    db.anime_ranking_table.aggregate([
        {"$match": {"title": "Trigun"}},
        {"$lookup": {
            "from": "demo_l",
            "localField": "demo_id",
            "foreignField": "demo_id",
            "as": "demo_data"}},
        {"$unwind": {
            "path": "$demo_data",
            "preserveNullAndEmptyArrays": True}},
        {"$group": {
            "_id": "$title",
            "demo_de": {"$first": "$demo_data.demo_de"}}},
        {"$project": {
            "_id": 0,
            "title": "$_id",
            "demo_de": {"$ifNull": ["$demo_de", "No demographic data"]}}}
    ])
    return time.time() - start_time

# Mesurer les temps d'exécution pour la requête 4
execution_times_query_4 = []
for _ in range(10):
    execution_time = execute_query_4()
    execution_times_query_4.append(execution_time)

# Retirer le max et le min pour la requête 4
execution_times_query_4.remove(max(execution_times_query_4))
execution_times_query_4.remove(min(execution_times_query_4))

# Calculer la moyenne pour la requête 1
average_time_query_4 = sum(execution_times_query_4) / len(execution_times_query_4)
average_time_query_4 = round(average_time_query_4, 3)
print(f"Average execution time for query 4: {average_time_query_4} seconds")

Average execution time for query 4: 0.005 seconds


## Data Analysis Queries

### Query 5

In [36]:
def execute_and_print_query_5():
    results = db.anime_ranking_table.aggregate([
        {"$lookup": {
            "from": "demo_l",
            "localField": "demo_id",
            "foreignField": "demo_id",
            "as": "demo_data"}},
        {"$unwind": "$demo_data"},
        {"$group": {
            "_id": "$demo_data.demo_de",
            "averagePopularity": {"$avg": "$popularity"}}},
        {"$project": {
            "_id": 0,
            "Type": "$_id",
            "Average_Popularity": {"$round": ["$averagePopularity", 2]}}},
        {"$sort": {"averagePopularity": -1}}
    ])
    for result in results:
        print(result)
execute_and_print_query_5()


{'Type': 'Shounen', 'Average_Popularity': 1625.98}
{'Type': 'Seinen', 'Average_Popularity': 1601.88}
{'Type': 'Kids', 'Average_Popularity': 4514.74}
{'Type': 'Josei', 'Average_Popularity': 1917.42}
{'Type': 'Shoujo', 'Average_Popularity': 1752.13}


In [46]:
# Query 5 Execution Function
def execute_query_5():
    start_time = time.time()
    db.anime_ranking_table.aggregate([
        {"$lookup": {
            "from": "demo_l",
            "localField": "demo_id",
            "foreignField": "demo_id",
            "as": "demo_data"}},
        {"$unwind": "$demo_data"},
        {"$group": {
            "_id": "$demo_data.demo_de",
            "averagePopularity": {"$avg": "$popularity"}}},
        {"$project": {
            "_id": 0,
            "Type": "$_id",
            "Average_Popularity": {"$round": ["$averagePopularity", 2]}}},
        {"$sort": {"averagePopularity": -1}}
    ])
    return time.time() - start_time

# Mesurer les temps d'exécution pour la requête 5
execution_times_query_5 = []
for _ in range(10):
    execution_time = execute_query_5()
    execution_times_query_5.append(execution_time)

# Retirer le max et le min pour la requête 5
execution_times_query_5.remove(max(execution_times_query_5))
execution_times_query_5.remove(min(execution_times_query_5))

# Calculer la moyenne pour la requête 5
average_time_query_5 = sum(execution_times_query_5) / len(execution_times_query_5)
average_time_query_5 = round(average_time_query_5, 2)
print(f"Average execution time for query 5: {average_time_query_5} seconds")

Average execution time for query 5: 0.24 seconds


### Query 6

In [38]:
def execute_and_print_query_6():
    results = db.anime_ranking_table.aggregate([
        {"$group": {
            "_id": "$studio_id",
            "averageRank": {"$avg": "$rank"}}},
        {"$sort": {"averageRank": 1}},
        {"$limit": 5},
        {"$lookup": {
            "from": "studios_l",
            "localField": "_id",
            "foreignField": "studio_id",
            "as": "studio_info"}},
        {"$unwind": "$studio_info"},
        {"$group": {
            "_id": "$studio_info.studio_de",
            "averageRank": {"$first": "$averageRank"}}},
        {"$sort": {"averageRank": 1}},
        {"$project": {
            "_id": 0,
            "studio": "$_id",
            "averageRank": {"$round": ["$averageRank", 2]}}}
    ])
    for result in results:
        print(result)
execute_and_print_query_6()

{'studio': 'Shuka', 'averageRank': 74.77}
{'studio': 'ufotable', 'averageRank': 96.23}
{'studio': 'Kyoto Animation', 'averageRank': 131.31}
{'studio': 'Orange', 'averageRank': 166.23}
{'studio': 'CloverWorks', 'averageRank': 207.81}


In [39]:
# Query 6 Execution Function
def execute_query_6():
    start_time = time.time()
    db.anime_ranking_table.aggregate([
        {"$group": {
            "_id": "$studio_id",
            "averageRank": {"$avg": "$rank"}}},
        {"$sort": {"averageRank": 1}},
        {"$limit": 5},
        {"$lookup": {
            "from": "studios_l",
            "localField": "_id",
            "foreignField": "studio_id",
            "as": "studio_info"}},
        {"$unwind": "$studio_info"},
        {"$group": {
            "_id": "$studio_info.studio_de",
            "averageRank": {"$first": "$averageRank"}}},
        {"$sort": {"averageRank": 1}},
        {"$project": {
            "_id": 0,
            "studio": "$_id",
            "averageRank": {"$round": ["$averageRank", 2]}}}
    ])
    return time.time() - start_time

# Measure the execution times for query 6
execution_times_query_6 = []
for _ in range(10):
    execution_time = execute_query_6()
    execution_times_query_6.append(execution_time)

# Remove the maximum and minimum values for query 6
execution_times_query_6.remove(max(execution_times_query_6))
execution_times_query_6.remove(min(execution_times_query_6))

# Calculate the average for query 6
average_time_query_6 = sum(execution_times_query_6) / len(execution_times_query_6)
average_time_query_6 = round(average_time_query_6, 2)
print(f"Average execution time for query 6: {average_time_query_6} seconds")

Average execution time for query 6: 0.01 seconds


### Query 7

In [40]:
def execute_and_print_query_7():
    results = db.anime_ranking_table.aggregate([
        {"$lookup": {
            "from": "anime_table",
            "localField": "mal_id",
            "foreignField": "mal_id",
            "as": "anime_details"}},
        {"$unwind": "$anime_details"},
        {"$group": {
            "_id": {
                "year": "$anime_details.start_season.year",
                "season": "$anime_details.start_season.season"},
            "average_popularity": {"$avg": "$popularity"}}},
        {"$project": {
            "_id": 0,
            "year": "$_id.year",
            "season": "$_id.season",
            "average_popularity": {"$round": ["$average_popularity", 2]}}},
        {"$sort": {"year": 1, "season": 1}}
    ])
    for result in results:
        print(result)
execute_and_print_query_7()

{'year': 1967.0, 'season': 'spring', 'average_popularity': 6349.42}
{'year': 1978.0, 'season': 'fall', 'average_popularity': 9719.92}
{'year': 1979.0, 'season': 'fall', 'average_popularity': 9791.23}
{'year': 1980.0, 'season': 'summer', 'average_popularity': 9789.85}
{'year': 1981.0, 'season': 'fall', 'average_popularity': 5696.92}
{'year': 1981.0, 'season': 'spring', 'average_popularity': 5311.0}
{'year': 1984.0, 'season': 'fall', 'average_popularity': 1537.31}
{'year': 1985.0, 'season': 'fall', 'average_popularity': 7956.08}
{'year': 1986.0, 'season': 'fall', 'average_popularity': 1148.38}
{'year': 1986.0, 'season': 'winter', 'average_popularity': 139.46}
{'year': 1988.0, 'season': 'spring', 'average_popularity': 4526.92}
{'year': 1989.0, 'season': 'spring', 'average_popularity': 102.15}
{'year': 1990.0, 'season': 'winter', 'average_popularity': 5175.92}
{'year': 1992.0, 'season': 'fall', 'average_popularity': 280.92}
{'year': 1992.0, 'season': 'spring', 'average_popularity': 556.85}

In [41]:
def execute_query_7():
    start_time = time.time()
    db.anime_ranking_table.aggregate([
        {"$lookup": {
            "from": "anime_table",
            "localField": "mal_id",
            "foreignField": "mal_id",
            "as": "anime_details"}},
        {"$unwind": "$anime_details"},
        {"$group": {
            "_id": {
                "year": "$anime_details.start_season.year",
                "season": "$anime_details.start_season.season"},
            "average_popularity": {"$avg": "$popularity"}}},
        {"$project": {
            "_id": 0,
            "year": "$_id.year",
            "season": "$_id.season",
            "average_popularity": {"$round": ["$average_popularity", 2]}}},
        {"$sort": {"year": 1, "season": 1}}
    ])
    return time.time() - start_time

# Measure the execution times for query 7
execution_times_query_7 = []
for _ in range(10):
    execution_time = execute_query_7()
    execution_times_query_7.append(execution_time)

# Remove the maximum and minimum values for query 7
execution_times_query_7.remove(max(execution_times_query_7))
execution_times_query_7.remove(min(execution_times_query_7))

# Calculate the average for query 7
average_time_query_7 = sum(execution_times_query_7) / len(execution_times_query_7)
average_time_query_7 = round(average_time_query_7, 2)
print(f"Average execution time for query 7: {average_time_query_7} seconds")

Average execution time for query 7: 11.4 seconds


### Query 8

In [42]:
def execute_and_print_query_8():
    results = db.anime_table.aggregate([
        {"$lookup": {
            "from": "genres_l",
            "localField": "genres_id",
            "foreignField": "genres_id",
            "as": "genres_data"}},
        {"$unwind": "$genres_data"},
        {"$lookup": {
            "from": "anime_ranking_table",
            "localField": "mal_id",
            "foreignField": "mal_id",
            "as": "ranking_data"}},
        {"$unwind": "$ranking_data"},
        {"$group": {
            "_id": "$genres_data.genres_de",
            "episodesCompletedPerUser": {
                "$avg": {
                    "$multiply": [
                        {"$divide": ["$ranking_data.statistics_completed", "$ranking_data.statistics_num_scoring_users"]},
                        100
                    ]
                }
            },
            "episodesDroppedPerUser": {
                "$avg": {
                    "$multiply": [
                        {"$divide": ["$ranking_data.statistics_dropped", "$ranking_data.statistics_num_scoring_users"]},
                        100
                    ]
                }
            }
        }},
        {"$project": {
            "_id": 0,
            "genre": "$_id",
            "episodesCompletedPerUser": {"$concat": [{"$toString": {"$round": ["$episodesCompletedPerUser", 2]}}, "%"]},
            "episodesDroppedPerUser": {"$concat": [{"$toString": {"$round": ["$episodesDroppedPerUser", 2]}}, "%"]}
        }}
    ])
    for result in results:
        print(result)
execute_and_print_query_8()

{'genre': 'Action', 'episodesCompletedPerUser': '59.44%', 'episodesDroppedPerUser': '5.01%'}
{'genre': 'Martial Arts', 'episodesCompletedPerUser': '64.99%', 'episodesDroppedPerUser': '8.21%'}
{'genre': 'Mecha', 'episodesCompletedPerUser': '39.46%', 'episodesDroppedPerUser': '7.72%'}
{'genre': 'Sports', 'episodesCompletedPerUser': '52.81%', 'episodesDroppedPerUser': '7.41%'}
{'genre': 'Slice of Life', 'episodesCompletedPerUser': '47.48%', 'episodesDroppedPerUser': '5.29%'}
{'genre': 'Mystery', 'episodesCompletedPerUser': '54.6%', 'episodesDroppedPerUser': '5.1%'}
{'genre': 'Cars', 'episodesCompletedPerUser': '46.07%', 'episodesDroppedPerUser': '9.36%'}
{'genre': 'Sci-Fi', 'episodesCompletedPerUser': '37.24%', 'episodesDroppedPerUser': '8.93%'}
{'genre': 'School', 'episodesCompletedPerUser': '59.27%', 'episodesDroppedPerUser': '4.59%'}
{'genre': 'Psychological', 'episodesCompletedPerUser': '79.61%', 'episodesDroppedPerUser': '0.8%'}
{'genre': 'Supernatural', 'episodesCompletedPerUser': '

In [43]:
# Query 4 Execution Function
def execute_query_8():
    start_time = time.time()
    db.anime_table.aggregate([
        {"$lookup": {
            "from": "genres_l",
            "localField": "genres_id",
            "foreignField": "genres_id",
            "as": "genres_data"}},
        {"$unwind": "$genres_data"},
        {"$lookup": {
            "from": "anime_ranking_table",
            "localField": "mal_id",
            "foreignField": "mal_id",
            "as": "ranking_data"}},
        {"$unwind": "$ranking_data"},
        {"$group": {
            "_id": "$genres_data.genres_de",
            "episodesCompletedPerUser": {
                "$avg": {
                    "$multiply": [
                        {"$divide": ["$ranking_data.statistics_completed", "$ranking_data.statistics_num_scoring_users"]},
                        100
                    ]
                }
            },
            "episodesDroppedPerUser": {
                "$avg": {
                    "$multiply": [
                        {"$divide": ["$ranking_data.statistics_dropped", "$ranking_data.statistics_num_scoring_users"]},
                        100
                    ]
                }
            }
        }},
        {"$project": {
            "_id": 0,
            "genre": "$_id",
            "episodesCompletedPerUser": {"$concat": [{"$toString": {"$round": ["$episodesCompletedPerUser", 2]}}, "%"]},
            "episodesDroppedPerUser": {"$concat": [{"$toString": {"$round": ["$episodesDroppedPerUser", 2]}}, "%"]}
        }}
    ])
    return time.time() - start_time

# Measure the execution times for query 8
execution_times_query_8 = []
for _ in range(10):
    execution_time = execute_query_8()
    execution_times_query_8.append(execution_time)

# Remove the maximum and minimum values for query 8
execution_times_query_8.remove(max(execution_times_query_8))
execution_times_query_8.remove(min(execution_times_query_8))

# Calculate the average for query 8
average_time_query_8 = sum(execution_times_query_8) / len(execution_times_query_8)
average_time_query_8 = round(average_time_query_8, 2)
print(f"Average execution time for query 8: {average_time_query_8} seconds")

Average execution time for query 8: 123.98 seconds


## Import execution time

In [62]:
queries_performance_data = [
    {"Query": "Fetch popular 'Adventure' anime with a popularity over 500", "Average Execution Time (seconds)": average_time_query_1},
    {"Query": "Retrieve all 'Madhouse' studio anime titles that have finished airing", "Average Execution Time (seconds)": average_time_query_2},
    {"Query": "Retrieve and display the rank and popularity of specific anime titles", "Average Execution Time (seconds)": average_time_query_3},
    {"Query": "Check for demographic information availability for a specific anime", "Average Execution Time (seconds)": average_time_query_4},
    {"Query": "Average Popularity Of Anime Within Each Type", "Average Execution Time (seconds)": average_time_query_5},
    {"Query": "Top 5 Studios by Average Anime Rankings", "Average Execution Time (seconds)": average_time_query_6},
    {"Query": "Analyze Anime Title Popularity Over Time by Season", "Average Execution Time (seconds)": average_time_query_7},
    {"Query": "Calculate Average Episodes Stats by Anime Genre per User", "Average Execution Time (seconds)": average_time_query_8}]

In [63]:
queries_performance_data

[{'Query': "Fetch popular 'Adventure' anime with a popularity over 500",
  'Average Execution Time (seconds)': 0.006},
 {'Query': "Retrieve all 'Madhouse' studio anime titles that have finished airing",
  'Average Execution Time (seconds)': 0.52},
 {'Query': 'Retrieve and display the rank and popularity of specific anime titles',
  'Average Execution Time (seconds)': 0.006},
 {'Query': 'Check for demographic information availability for a specific anime',
  'Average Execution Time (seconds)': 0.005},
 {'Query': 'Average Popularity Of Anime Within Each Type',
  'Average Execution Time (seconds)': 0.24},
 {'Query': 'Top 5 Studios by Average Anime Rankings',
  'Average Execution Time (seconds)': 0.01},
 {'Query': 'Analyze Anime Title Popularity Over Time by Season',
  'Average Execution Time (seconds)': 11.4},
 {'Query': 'Calculate Average Episodes Stats by Anime Genre per User',
  'Average Execution Time (seconds)': 123.98}]

In [64]:
df = pd.DataFrame(queries_performance_data)
csv_file = "queries_performance.csv"
df.to_csv(csv_file, index=False)